<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Pipeline

**Tags:** #naas #pipeline #jupyter #notebook #dataanalysis #workflow #streamline

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Description:** This notebook creates a notebooks pipeline.

## Input

### Import libraries

In [1]:
from naas.pipeline import (
    Pipeline,
    NotebookStep,
    End,
    ParallelStep,
)
from datetime import date
import os
import glob
import naas
import naas_data_product

✅ utils file '/home/ftp/abi/utils/data.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_chat_plugin.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_lab.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/openai.ipynb' successfully loaded.


### Setup variables

In [2]:
# Datalake
datalake_dir = os.path.join("/", "home", "ftp", "abi", "outputs")

# LinkedIn
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_LINKEDIN_LI_AT" #example: AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_LINKEDIN_JSESSIONID" #example: ajax:8379907400220387585
linkedin_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "linkedin_url") or "YOUR_LINKEDIN_URL"

# Google Sheets
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_growth = "GROWTH"
sheet_leads = "LEADS"
sheet_leads_companies = "LEADS_COMPANIES"
sheet_sales = "SALES"

# OpenAI API key
openai_api_key = naas.secret.get("OPENAI_API_KEY") or "YOUR_OPENAI_API_KEY"

# Notification
email_to = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "emails") or []

# Scheduler
cron = "0 9 * * *"

## Model

### Create output directory by date

In [3]:
sales_dir = os.path.join(datalake_dir, "sales-engine", date.today().isoformat())
print("✅ Output directory:", sales_dir)

✅ Output directory: /home/ftp/abi/outputs/sales-engine/2023-12-19


### Setup notebooks
This section is made to declare all the notebooks and create unique IDs that will be used in the pipeline. 

In [4]:
sales = NotebookStep(
    name="Sales Qualified Leads",
    notebook_path="pipeline_templates/Google_Sheets_Update_sales_db.ipynb",
    parameters={
        "openai_api_key": openai_api_key,
        "spreadsheet_url": spreadsheet_url,
        "leads_companies_name": sheet_leads_companies,
        "leads_profiles_name": sheet_leads,
        "sheet_name_output": sheet_sales,
        "output_dir": sales_dir,
    }
)
analytics = NotebookStep(
    name="Create analytics",
    notebook_path="pipeline_templates/Plotly_Create_analytics.ipynb",
    parameters={
        "input_dir": sales_dir,
        "output_dir": sales_dir,
    }
)
plugin = NotebookStep(
    name="Generate plugin",
    notebook_path="__plugin__.ipynb",
    parameters={
        "input_dir": sales_dir,
        "spreadsheet_url": spreadsheet_url,
        "sheet_name": sheet_leads,
    }
)
email = NotebookStep(
    name="Send Email notification",
    notebook_path="pipeline_templates/Naas_Send_notification.ipynb",
    parameters={
        "input_dir": sales_dir,
        "datalake_dir": datalake_dir,
        "email_to": email_to,
    }
)

## Output

### Run pipeline

In [5]:
pipeline = Pipeline()

pipeline >> sales >> analytics >> plugin >> email >> End()

pipeline.run(outputs_path=os.path.join(sales_dir, "pipeline_executions"))

### Add dependencies

In [6]:
pipeline_templates = glob.glob("pipeline_templates/*.ipynb")
utils_files = glob.glob(f"{naas_data_product.UTILS_PATH}/*.ipynb")
files = pipeline_templates + utils_files + ["__plugin__.ipynb"]

for file in files: 
    naas.dependency.add(file)

👌 Well done! Your Dependency /home/ftp/abi/models/sales-engine/pipeline_templates/Naas_Send_notification.ipynb has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
👌 Well done! Your Dependency /home/ftp/abi/models/sales-engine/pipeline_templates/Google_Sheets_Update_sales_db.ipynb has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
👌 Well done! Your Dependency /home/ftp/abi/models/sales-engine/pipeline_templates/Plotly_Create_analytics.ipynb has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
👌 Well done! Your Dependency /home/ftp/abi/utils/naas_chat_plugin.ipynb has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
👌 Well done! Your Dependency /home/ftp/abi/utils/openai.ipynb has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
👌 Well done! Your Depende

### Add scheduler

In [7]:
naas.scheduler.add(cron=cron)

# Uncomment the line below to delete your automation
# naas.scheduler.delete()

👌 Well done! Your Notebook has been sent to production.

⏰ It will be scheduled "At 09:00 every day" (more on the syntax on https://crontab.guru/).

Ps: to remove the "Scheduler", just replace .add by .delete
